In [1]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import *
import numpy as np
from tqdm import tqdm
import os
import matplotlib.pyplot as plt
import cv2
from gait import *
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from utils import angle_ims, get_device

In [2]:
os.chdir('/home/ishikaa/Downloads/')
labels = fetch_labels(label_angle='000',save=False,override=True)
files = [filename for filename, value in labels.items() if value==4]
images_0_4 = [preprocess(cv2.imread(file))/255 for file in files]
y = np.mean([image for image in images_0_4],axis=0)

100%|██████████| 124/124 [00:00<00:00, 493.04it/s]


In [3]:
class Conv3D(nn.Module):

    '''
    Performs 3D convolution
    '''

    def __init__(self):

        super(Conv3D, self).__init__()
        self.conv1 = self._convblock(1,16,3,1,False)
        self.conv2 = self._convblock(16,32,1,1,False)
        self.conv3 = self._convblock(32,16,1,1,False)
        self.conv4 = self._convblock(16,1,1,1,True)

    def _convblock(self,in_channels,out_channels,ksized,ksize,last):
        '''
        Makes a block of layers (Conv3d,ReLU,Maxpool3d,BatchNorm3d(only if !last))
        '''

        l1 = nn.Conv3d(in_channels=in_channels,out_channels=out_channels,kernel_size=(ksized,ksize,ksize))
        l2 = nn.ReLU()
        
        if last:
            return nn.Sequential(l1,l2)
        else:
            l3 = nn.MaxPool3d((1, 1, 1))
            l4 = nn.BatchNorm3d(out_channels)
            return nn.Sequential(l1,l2,l3,l4)
        
        
    def forward(self,x):

        out = self.conv1(x)
        out = self.conv2(out)
        out = self.conv3(out)
        out = self.conv4(out)
        
        return out


In [4]:
model = Conv3D()

In [6]:
model = Conv3D()
device = get_device()
print(device)
model.to(device)

cuda:0


Conv3D(
  (conv1): Sequential(
    (0): Conv3d(1, 16, kernel_size=(3, 1, 1), stride=(1, 1, 1))
    (1): ReLU()
    (2): MaxPool3d(kernel_size=(1, 1, 1), stride=(1, 1, 1), padding=0, dilation=1, ceil_mode=False)
    (3): BatchNorm3d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv2): Sequential(
    (0): Conv3d(16, 32, kernel_size=(1, 1, 1), stride=(1, 1, 1))
    (1): ReLU()
    (2): MaxPool3d(kernel_size=(1, 1, 1), stride=(1, 1, 1), padding=0, dilation=1, ceil_mode=False)
    (3): BatchNorm3d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv3): Sequential(
    (0): Conv3d(32, 16, kernel_size=(1, 1, 1), stride=(1, 1, 1))
    (1): ReLU()
    (2): MaxPool3d(kernel_size=(1, 1, 1), stride=(1, 1, 1), padding=0, dilation=1, ceil_mode=False)
    (3): BatchNorm3d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv4): Sequential(
    (0): Conv3d(16, 1, kernel_size=(1, 1, 1), stride=(1, 1, 1))
    (1): ReLU()

In [12]:
class PEIData(Dataset):

    def __init__(self, num_exps,transform=None):
        """
        Custom dataset for images of a certain keypose at a given angle.

        Args:
            angle (int)     : Angle
            keypose (int)   : Key-pose/Cluster
            data_path (str) : Path where dataset is downloaded
        """
        
        ds = []
        for i in range(num_exps):
            exp = angle_ims(exp=i+1,angle=0,keypose = 4)
            ds = ds + exp

        images = np.empty((len(ds),3,ds[0].shape[2]//2,ds[0].shape[1]//2))

        for i in range(len(ds)):
            images[i] = np.asarray([preprocess(im)/255 for im in ds[i]])
        
        self.images = images.reshape(images.shape[0],1,images.shape[1],images.shape[2],images.shape[3]).astype('float32')

        #Avg PEI after PCA .
        # self.y = np.mean([image for image in images_0_4],axis=0)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        return self.images[idx]

In [13]:
data = PEIData(2)

In [25]:
len(exp1)+len(exp2)

14

In [35]:
total = []
for i in range(1,3):
    exp,_ = angle_ims(exp=i,angle=0,keypose = 4)
    total = total + exp

In [47]:
total[0][0].shape

(240, 320, 3)

## Training

In [79]:
dl = DataLoader(data, 1, False)
#Loss
criterion = nn.MSELoss()
#Optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
batch = 8
mean_frame = torch.from_numpy(mean_frame).to(device,dtype=torch.float)

In [80]:
data[0].dtype

torch.float32

In [85]:
n_epochs = 3

for epoch in range(1, n_epochs+1):
    # monitor training loss
    train_loss = 0.0

    #Training
    for data in dl:
        images = data
        images = images.to(device)
        optimizer.zero_grad()
        out = model(images)
        temp_out = torch.reshape(out, (out.shape[3],out.shape[4]))
        # print('Outputs: ',temp_out.dtype,' Targets shape: ',mean_frame.dtype)
        loss = criterion(temp_out, mean_frame)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()*images.size(0)
          
    train_loss = train_loss/len(dl)
    print('Epoch: {} \tTraining Loss: {:.6f}'.format(epoch, train_loss))

Epoch: 1 	Training Loss: 0.272077
Epoch: 2 	Training Loss: 0.272077
Epoch: 3 	Training Loss: 0.272077


In [66]:
dataiter = iter(test_dl)
images = dataiter.next()

torch.Size([1, 160, 120])

In [61]:
out.shape

torch.Size([1, 1, 1, 160, 120])

In [63]:
(out.shape[0],out.shape[2],out.shape[3])

(1, 1, 160)

## USE FOR Y

/home/ishikaa/Desktop/Repos/Gait-Occlusion-GAN


In [14]:
cv2.imshow('',mean_frame)
cv2.waitKey(0)
cv2.destroyAllWindows()

## Encoder testings
input:  `<class 'torch.Tensor'> torch.Size([m, 1, 160, 120])`

In [16]:
from encoder import Autoencoder